# Running drivers

Drivers can be run either from **bash** using the `uw` command-line interface (CLI), or from **Python** using the application programming interface (API) provided by the `uwtools.api` namespace. Below, we demonstrate running the prep utility `chgres_cube` via the API, but otherwise use the CLI.

In [ ]:
# Set up the environment using Python so it's available to each bash subshell below, along with all the Python cells.

import os
from datetime import datetime, timedelta
from pathlib import Path
uw_training = "/scratch3/BMC/wrfruc/cholt/uw_training_prep/uw_training"
os.environ["uw_training"] = uw_training

cycle = datetime(2025, 8, 27, 12)

## Run chgres_cube for initial conditions

In [ ]:
%%script bash -l
args=(
    -c $uw_training/configs/prepare_bcs.yaml
    --key-path make_ics
    --cycle 2025-08-27T12
    --leadtime 0
    --batch
)
uw chgres_cube run ${args[*]}

### A Python API alternative for chgres_cube

Docs: __[uwtools.api.chgres_cube](https://uwtools.readthedocs.io/en/main/sections/user_guide/api/chgres_cube.html)__

This is applicable to all drivers.

In [ ]:
from uwtools.api.chgres_cube import ChgresCube
from uwtools.api.logging import use_uwtools_logger

use_uwtools_logger()

ics = ChgresCube(
    config=Path(uw_training, "configs", "prepare_bcs.yaml"),
    cycle=cycle,
    leadtime=timedelta(hours=0),
    key_path=["make_ics"],
)
ics.provisioned_rundir()

In [ ]:
help(ics)

In [ ]:
ics.config

In [ ]:
ics.output

## Run chgres_cube for lateral boundary conditions

Run once for each lead time in a loop. Always point to the same section of the UW YAML.

In [ ]:
%%script bash -l
for i in $(seq 0 6) ; do 
  args=(
      -c $uw_training/configs/prepare_bcs.yaml
      --key-path make_lbcs
      --cycle 2025-08-27T12
      --leadtime $i
      --batch
  )
  uw chgres_cube run ${args[*]}
done

## Run a cold-start FV3 forecast

In [ ]:
%%script bash -l
args=(
  -c $uw_training/configs/fv3_config.yaml
  --key-path forecast_prod
  --cycle 2025-08-27T12
  --batch 
)
uw fv3 run ${args[*]}

## Run the analysis at the *NEXT* cycle

Here, the cycling frequency is 1 hour.

In [ ]:
%%script bash -l
# The notebook has a hard time displaying the output in a timely manner for the number of links required by GSI. Pipe to a file on disk.
args=(
    -c $uw_training/configs/gsi_uw_config.yaml
    --key-path analysis
    --cycle 2025-08-27T13
    --batch
)
uw gsi run ${args[*]} > gsi.out 2>&1

In [ ]:
%%bash
cat gsi.out

## Run a cycled FV3 forecast from the updated analysis 

* This forecast was configured to run using the lateral boundary conditions from the previous cycle.
* It runs a script that removes the checksums from the analysis-update files between staging the files and submitting the job.

In [ ]:
%%bash
cat $uw_training/scripts/run_forecast.sh

In [ ]:
%%script bash -l
$uw_training/scripts/run_forecast.sh $uw_training/configs/fv3_config.yaml 2025-08-27T13